# Preprocess ERA5 Reanalysis

This notebook preprocesses ERA5 reanalysis so it's easier to compare against observations at the ENA site. This means:
* renaming awkward variable names from the grib-to-netcdf conversion
* calculating pressure at levels and interfaces
* calculating geopotential height (needed because obs are in height coordinates)

Contact: christopher.jones@pnnl.gov

In [1]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

In [14]:
# input_file_name = '/Users/jone003/tmp/EAGLES/ERA5_ENA_20170717-20170719_ml.nc'
input_file_name = '/Users/jone003/tmp/EAGLES/ERA5_ENA_20180116-20180126_ml.nc'
output_file_name = input_file_name.replace('_ml.nc', '_with_height_and_presl.nc')

## Load dataset and rename variables
Note: ERA5 has already been converted from `grib` to `netcdf` format with `ncl_convert2nc`
* Convert ERA5 mode levels to pressure and geopotential height
* (optional) interpolate time and heights between ERA5 and obs for easier comparison

In [2]:
def fix_dumb_names(ds, postfix='_P0_L105_GLL0', **kwargs):
    """removes suffixes from grib to netcdf conversion
    optionally rename (orig: new) in kwargs
    """
    to_rename = kwargs.copy()
    for v in ds:
        if postfix in v:
            to_rename.update({v: v.replace(postfix, '')})
            print(v, v.replace(postfix,''))
    print(to_rename)
    return ds.rename(to_rename)

In [3]:
ds_era = xr.open_dataset(input_file_name)
ds_era = fix_dumb_names(ds_era, lat_0='lat', lon_0='lon', lv_HYBL0='lev', initial_time0_hours='time')
print(ds_era)

TMP_P0_L105_GLL0 TMP
SPFH_P0_L105_GLL0 SPFH
UGRD_P0_L105_GLL0 UGRD
VGRD_P0_L105_GLL0 VGRD
GP_P0_L105_GLL0 GP
NLPRES_P0_L105_GLL0 NLPRES
{'lat_0': 'lat', 'lon_0': 'lon', 'lv_HYBL0': 'lev', 'initial_time0_hours': 'time', 'TMP_P0_L105_GLL0': 'TMP', 'SPFH_P0_L105_GLL0': 'SPFH', 'UGRD_P0_L105_GLL0': 'UGRD', 'VGRD_P0_L105_GLL0': 'VGRD', 'GP_P0_L105_GLL0': 'GP', 'NLPRES_P0_L105_GLL0': 'NLPRES'}
<xarray.Dataset>
Dimensions:                (lat: 41, lev: 137, lon: 61, time: 264)
Coordinates:
  * time                   (time) datetime64[ns] 2018-01-16 ... 2018-01-26T23:00:00
  * lat                    (lat) float32 45.0 44.75 44.5 ... 35.5 35.25 35.0
  * lon                    (lon) float32 325.0 325.25 325.5 ... 339.75 340.0
  * lev                    (lev) float32 1.0 2.0 3.0 4.0 ... 135.0 136.0 137.0
Data variables:
    TMP                    (time, lev, lat, lon) float32 ...
    SPFH                   (time, lev, lat, lon) float32 ...
    UGRD                   (time, lev, lat, lon) float32 

## Convert from model level to pressure and geopotential height
To convert from level to geopotential height, follow procedure outlined here
         https://confluence.ecmwf.int/pages/viewpage.action?pageId=88261819

The essence of the calculation is:
    # compute the pressures (on half-levels)
    ph_lev, ph_levplusone = get_ph_levs(values, lev)
 
    if lev == 1:
        dlog_p = np.log(ph_levplusone / 0.1)
        alpha = np.log(2)
    else:
        dlog_p = np.log(ph_levplusone / ph_lev)
        alpha = 1. - ((ph_lev / (ph_levplusone - ph_lev)) * dlog_p)
 
    t_level = t_level * R_D
 
    # z_f is the geopotential of this full level
    # integrate from previous (lower) half-level z_h to the
    # full level
    z_f = z_h + (t_level * alpha)
 
    # z_h is the geopotential of 'half-levels'
    # integrate z_h to next half level
    z_h = z_h + (t_level * dlog_p)
 
    return z_h, z_f

In [4]:
# source: https://www.ecmwf.int/en/forecasts/documentation-and-support/137-model-levels
model_level_table = """
0	0.000000	0.000000	0.000000	-  	-	-	-	-
1	2.000365	0.000000	0.0200	0.0100	79301.79	80301.65	198.05	0.000018
2	3.102241	0.000000	0.0310	0.0255	73721.58	74584.91	209.21	0.000042
3	4.666084	0.000000	0.0467	0.0388	71115.75	71918.79	214.42	0.000063
4	6.827977	0.000000	0.0683	0.0575	68618.43	69365.77	221.32	0.000090
5	9.746966	0.000000	0.0975	0.0829	66210.99	66906.53	228.06	0.000127
6	13.605424	0.000000	0.1361	0.1168	63890.03	64537.43	234.56	0.000173
7	18.608931	0.000000	0.1861	0.1611	61651.77	62254.39	240.83	0.000233
8	24.985718	0.000000	0.2499	0.2180	59492.50	60053.46	246.87	0.000308
9	32.985710	0.000000	0.3299	0.2899	57408.61	57930.78	252.71	0.000400
10	42.879242	0.000000	0.4288	0.3793	55396.62	55882.68	258.34	0.000512
11	54.955463	0.000000	0.5496	0.4892	53453.20	53905.62	263.78	0.000646
12	69.520576	0.000000	0.6952	0.6224	51575.15	51996.21	269.04	0.000806
13	86.895882	0.000000	0.8690	0.7821	49767.41	50159.36	270.65	0.001007
14	107.415741	0.000000	1.0742	0.9716	48048.70	48413.94	270.65	0.001251
15	131.425507	0.000000	1.3143	1.1942	46416.22	46756.98	269.02	0.001546
16	159.279404	0.000000	1.5928	1.4535	44881.17	45199.69	264.72	0.001913
17	191.338562	0.000000	1.9134	1.7531	43440.23	43738.55	260.68	0.002343
18	227.968948	0.000000	2.2797	2.0965	42085.00	42364.93	256.89	0.002843
19	269.539581	0.000000	2.6954	2.4875	40808.05	41071.20	253.31	0.003421
20	316.420746	0.000000	3.1642	2.9298	39602.76	39850.56	249.94	0.004084
21	368.982361	0.000000	3.6898	3.4270	38463.25	38696.94	246.75	0.004838
22	427.592499	0.000000	4.2759	3.9829	37384.22	37604.95	243.73	0.005693
23	492.616028	0.000000	4.9262	4.6010	36360.94	36569.72	240.86	0.006655
24	564.413452	0.000000	5.6441	5.2851	35389.15	35586.89	238.14	0.007731
25	643.339905	0.000000	6.4334	6.0388	34465.00	34652.52	235.55	0.008931
26	729.744141	0.000000	7.2974	6.8654	33585.02	33763.05	233.09	0.010261
27	823.967834	0.000000	8.2397	7.7686	32746.04	32915.27	230.74	0.011729
28	926.344910	0.000000	9.2634	8.7516	31945.53	32106.57	228.60	0.013337
29	1037.201172	0.000000	10.3720	9.8177	31177.59	31330.96	227.83	0.015012
30	1156.853638	0.000000	11.5685	10.9703	30438.54	30584.71	227.09	0.016829
31	1285.610352	0.000000	12.8561	12.2123	29726.69	29866.09	226.38	0.018793
32	1423.770142	0.000000	14.2377	13.5469	29040.48	29173.50	225.69	0.020910
33	1571.622925	0.000000	15.7162	14.9770	28378.46	28505.47	225.03	0.023186
34	1729.448975	0.000000	17.2945	16.5054	27739.29	27860.64	224.39	0.025624
35	1897.519287	0.000000	18.9752	18.1348	27121.74	27237.73	223.77	0.028232
36	2076.095947	0.000000	20.7610	19.8681	26524.63	26635.56	223.17	0.031013
37	2265.431641	0.000000	22.6543	21.7076	25946.90	26053.04	222.60	0.033972
38	2465.770508	0.000000	24.6577	23.6560	25387.55	25489.15	222.04	0.037115
39	2677.348145	0.000000	26.7735	25.7156	24845.63	24942.93	221.50	0.040445
40	2900.391357	0.000000	29.0039	27.8887	24320.28	24413.50	220.97	0.043967
41	3135.119385	0.000000	31.3512	30.1776	23810.67	23900.02	220.46	0.047685
42	3381.743652	0.000000	33.8174	32.5843	23316.04	23401.71	219.97	0.051604
43	3640.468262	0.000000	36.4047	35.1111	22835.68	22917.85	219.49	0.055727
44	3911.490479	0.000000	39.1149	37.7598	22368.91	22447.75	219.02	0.060059
45	4194.930664	0.000000	41.9493	40.5321	21915.16	21990.82	218.57	0.064602
46	4490.817383	0.000000	44.9082	43.4287	21473.98	21546.62	218.12	0.069359
47	4799.149414	0.000000	47.9915	46.4498	21045.00	21114.77	217.70	0.074330
48	5119.895020	0.000000	51.1990	49.5952	20627.87	20694.90	217.28	0.079516
49	5452.990723	0.000000	54.5299	52.8644	20222.24	20286.66	216.87	0.084916
50	5798.344727	0.000000	57.9834	56.2567	19827.95	19889.88	216.65	0.090458
51	6156.074219	0.000000	61.5607	59.7721	19443.55	19503.09	216.65	0.096110
52	6526.946777	0.000000	65.2695	63.4151	19068.35	19125.61	216.65	0.101968
53	6911.870605	0.000000	69.1187	67.1941	18701.27	18756.34	216.65	0.108045
54	7311.869141	0.000000	73.1187	71.1187	18341.27	18394.25	216.65	0.114355
55	7727.412109	0.000007	77.2810	75.1999	17987.41	18038.35	216.65	0.120917
56	8159.354004	0.000024	81.6182	79.4496	17638.78	17687.77	216.65	0.127751
57	8608.525391	0.000059	86.1450	83.8816	17294.53	17341.62	216.65	0.134877
58	9076.400391	0.000112	90.8774	88.5112	16953.83	16999.08	216.65	0.142321
59	9562.682617	0.000199	95.8280	93.3527	16616.09	16659.55	216.65	0.150106
60	10065.978516	0.000340	101.0047	98.4164	16281.10	16322.83	216.65	0.158248
61	10584.631836	0.000562	106.4153	103.7100	15948.85	15988.88	216.65	0.166760
62	11116.662109	0.000890	112.0681	109.2417	15619.30	15657.70	216.65	0.175655
63	11660.067383	0.001353	117.9714	115.0198	15292.44	15329.24	216.65	0.184946
64	12211.547852	0.001992	124.1337	121.0526	14968.24	15003.50	216.65	0.194646
65	12766.873047	0.002857	130.5637	127.3487	14646.68	14680.44	216.65	0.204770
66	13324.668945	0.003971	137.2703	133.9170	14327.75	14360.05	216.65	0.215331
67	13881.331055	0.005378	144.2624	140.7663	14011.41	14042.30	216.65	0.226345
68	14432.139648	0.007133	151.5493	147.9058	13697.65	13727.18	216.65	0.237825
69	14975.615234	0.009261	159.1403	155.3448	13386.45	13414.65	216.65	0.249786
70	15508.256836	0.011806	167.0450	163.0927	13077.79	13104.70	216.65	0.262244
71	16026.115234	0.014816	175.2731	171.1591	12771.64	12797.30	216.65	0.275215
72	16527.322266	0.018318	183.8344	179.5537	12467.99	12492.44	216.65	0.288713
73	17008.789063	0.022355	192.7389	188.2867	12166.81	12190.10	216.65	0.302755
74	17467.613281	0.026964	201.9969	197.3679	11868.08	11890.24	216.65	0.317357
75	17901.621094	0.032176	211.6186	206.8078	11571.79	11592.86	216.65	0.332536
76	18308.433594	0.038026	221.6146	216.6166	11277.92	11297.93	216.65	0.348308
77	18685.718750	0.044548	231.9954	226.8050	10986.70	11005.69	216.74	0.364545
78	19031.289063	0.051773	242.7719	237.3837	10696.22	10714.22	218.62	0.378253
79	19343.511719	0.059728	253.9549	248.3634	10405.61	10422.64	220.51	0.392358
80	19620.042969	0.068448	265.5556	259.7553	10114.89	10130.98	222.40	0.406868
81	19859.390625	0.077958	277.5852	271.5704	9824.08	9839.26	224.29	0.421790
82	20059.931641	0.088286	290.0548	283.8200	9533.20	9547.49	226.18	0.437130
83	20219.664063	0.099462	302.9762	296.5155	9242.26	9255.70	228.08	0.452897
84	20337.863281	0.111505	316.3607	309.6684	8951.30	8963.90	229.97	0.469097
85	20412.308594	0.124448	330.2202	323.2904	8660.32	8672.11	231.86	0.485737
86	20442.078125	0.138313	344.5663	337.3932	8369.35	8380.36	233.75	0.502825
87	20425.718750	0.153125	359.4111	351.9887	8078.41	8088.67	235.64	0.520367
88	20361.816406	0.168910	374.7666	367.0889	7787.51	7797.04	237.53	0.538370
89	20249.511719	0.185689	390.6450	382.7058	7496.68	7505.51	239.42	0.556842
90	20087.085938	0.203491	407.0583	398.8516	7205.93	7214.09	241.31	0.575790
91	19874.025391	0.222333	424.0190	415.5387	6915.29	6922.80	243.20	0.595219
92	19608.572266	0.242244	441.5395	432.7792	6624.76	6631.66	245.09	0.615138
93	19290.226563	0.263242	459.6321	450.5858	6334.38	6340.68	246.98	0.635553
94	18917.460938	0.285354	478.3096	468.9708	6044.15	6049.89	248.86	0.656471
95	18489.707031	0.308598	497.5845	487.9470	5754.10	5759.30	250.75	0.677899
96	18006.925781	0.332939	517.4198	507.5021	5464.60	5469.30	252.63	0.699815
97	17471.839844	0.358254	537.7195	527.5696	5176.77	5180.98	254.50	0.722139
98	16888.687500	0.384363	558.3430	548.0312	4892.26	4896.02	256.35	0.744735
99	16262.046875	0.411125	579.1926	568.7678	4612.58	4615.92	258.17	0.767472
100	15596.695313	0.438391	600.1668	589.6797	4338.77	4341.73	259.95	0.790242
101	14898.453125	0.466003	621.1624	610.6646	4071.80	4074.41	261.68	0.812937
102	14173.324219	0.493800	642.0764	631.6194	3812.53	3814.82	263.37	0.835453
103	13427.769531	0.521619	662.8084	652.4424	3561.70	3563.69	265.00	0.857686
104	12668.257813	0.549301	683.2620	673.0352	3319.94	3321.67	266.57	0.879541
105	11901.339844	0.576692	703.3467	693.3043	3087.75	3089.25	268.08	0.900929
106	11133.304688	0.603648	722.9795	713.1631	2865.54	2866.83	269.52	0.921768
107	10370.175781	0.630036	742.0855	732.5325	2653.58	2654.69	270.90	0.941988
108	9617.515625	0.655736	760.5996	751.3426	2452.04	2452.99	272.21	0.961527
109	8880.453125	0.680643	778.4661	769.5329	2260.99	2261.80	273.45	0.980334
110	8163.375000	0.704669	795.6396	787.0528	2080.41	2081.09	274.63	0.998368
111	7470.343750	0.727739	812.0847	803.8622	1910.19	1910.76	275.73	1.015598
112	6804.421875	0.749797	827.7756	819.9302	1750.14	1750.63	276.77	1.032005
113	6168.531250	0.770798	842.6959	835.2358	1600.04	1600.44	277.75	1.047576
114	5564.382813	0.790717	856.8376	849.7668	1459.58	1459.91	278.66	1.062310
115	4993.796875	0.809536	870.2004	863.5190	1328.43	1328.70	279.52	1.076209
116	4457.375000	0.827256	882.7910	876.4957	1206.21	1206.44	280.31	1.089286
117	3955.960938	0.843881	894.6222	888.7066	1092.54	1092.73	281.05	1.101558
118	3489.234375	0.859432	905.7116	900.1669	987.00	987.15	281.73	1.113047
119	3057.265625	0.873929	916.0815	910.8965	889.17	889.29	282.37	1.123777
120	2659.140625	0.887408	925.7571	920.9193	798.62	798.72	282.96	1.133779
121	2294.242188	0.899900	934.7666	930.2618	714.94	715.02	283.50	1.143084
122	1961.500000	0.911448	943.1399	938.9532	637.70	637.76	284.00	1.151724
123	1659.476563	0.922096	950.9082	947.0240	566.49	566.54	284.47	1.159733
124	1387.546875	0.931881	958.1037	954.5059	500.91	500.95	284.89	1.167147
125	1143.250000	0.940860	964.7584	961.4311	440.58	440.61	285.29	1.173999
126	926.507813	0.949064	970.9046	967.8315	385.14	385.16	285.65	1.180323
127	734.992188	0.956550	976.5737	973.7392	334.22	334.24	285.98	1.186154
128	568.062500	0.963352	981.7968	979.1852	287.51	287.52	286.28	1.191523
129	424.414063	0.969513	986.6036	984.2002	244.68	244.69	286.56	1.196462
130	302.476563	0.975078	991.0230	988.8133	205.44	205.44	286.81	1.201001
131	202.484375	0.980072	995.0824	993.0527	169.50	169.51	287.05	1.205168
132	122.101563	0.984542	998.8081	996.9452	136.62	136.62	287.26	1.208992
133	62.781250	0.988500	1002.2250	1000.5165	106.54	106.54	287.46	1.212498
134	22.835938	0.991984	1005.3562	1003.7906	79.04	79.04	287.64	1.215710
135	3.757813	0.995003	1008.2239	1006.7900	53.92	53.92	287.80	1.218650
136	0.000000	0.997630	1010.8487	1009.5363	30.96	30.96	287.95	1.221341
137	0.000000	1.000000	1013.2500	1012.0494	10.00	10.00	288.09	1.223803
"""

In [5]:
# extract a and b coefficients to convert from model levels to pressure
nlevel = []
acoef = []
bcoef = []
for row in model_level_table.split('\n'):
    if row:
        vals = row.split('\t')[:3]
        nlevel.append(int(vals[0]))
        acoef.append(float(vals[1]))
        bcoef.append(float(vals[2]))

nlev = np.array(nlevel)
ac = np.array(acoef)  # at half levels (interfaces)
bc = np.array(bcoef)  # at half levels (interfaces)

R_D = 287.06
R_G = 9.80665

da_hyba = xr.DataArray(data=ac, dims=['ilev'], coords=[nlev])
da_hybb = xr.DataArray(data=bc, dims=['ilev'], coords=[nlev])
presi = da_hyba + da_hybb * np.exp(ds_era['NLPRES'])  # pressure at interfaces
# moist temperature:
moist_temp = ds_era['TMP'] * (1 + 0.609133 * ds_era['SPFH'])

In [6]:
def calc_dlogp(presi):
    """calculate dlogp and alpha on interfaces (ilev)"""
    axis = presi.dims.index('ilev')  # should be 0
    if axis != 0:
        return 'wrong axis!'
    arr = presi.values
    
    ph_lev = arr[0:-1, ...]
    ph_levplusone = arr[1:, ...]
    
    # rows 1:end
    dlogp = np.log(ph_levplusone / ph_lev)
    dlogp[0, ...] = np.log(arr[1, ...] / 0.1)
    
    alpha = 1 - (ph_lev / (ph_levplusone - ph_lev)) * dlogp
    alpha[0, ...] = np.log(2)
    
    return dlogp, alpha

def calc_geopotential(presi, moist_temp, z0):
    """Returns geopotential height at interfaces (gzi), model levels (gz), and height dataArray (z)
    """
    R_D = 287.06
    R_G = 9.80665
    rhs = R_D * moist_temp.transpose('lev', 'time', 'lat', 'lon').values
    
    dlogp, alpha = calc_dlogp(presi)
    gzi = np.zeros_like(presi.values)
    gzi[0, ...] = z0.values.copy()
    gzi[1:, ...] = gzi[0, ...] + np.cumsum((rhs * dlogp)[::-1, ...], axis=0)  # reversed ...
    
    gzi = gzi[::-1, ...]
    gz = np.zeros_like(rhs)
    gz = gzi[1:, ...] + (rhs * alpha)
    
    the_dims=['lev', 'time', 'lat', 'lon']
    z = xr.DataArray(gz / (R_G * 1000), coords=[moist_temp[d] for d in the_dims], 
                     dims=the_dims,
                     name='height',
                     attrs={'units': 'km', 'long_name': 'height above sea level'})
    return gzi, gz, z

In [7]:
# calculate height and add to ds_era dataset
gzi, gz, z = calc_geopotential(presi, moist_temp, ds_era['GP'])
ds_era['height'] = z

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


In [8]:
# add pressure levels to the dataset
the_dims = ['lev', 'time', 'lat', 'lon']

# pressure levels are just taken as linear midpoint between interface pressures
presl_vals = (presi[:-1, ...].values + presi[1:, ...].values) / 2
presl = xr.DataArray(presl_vals, coords=[ds_era[d] for d in the_dims],
                     dims=the_dims, name='pressure',
                     attrs={'units': 'Pa', 'long_name': 'pressure on model levels'})
ds_era['pressure'] = presl
ds_era  # print to screen to verify contents

<xarray.Dataset>
Dimensions:                (lat: 41, lev: 137, lon: 61, time: 264)
Coordinates:
  * time                   (time) datetime64[ns] 2018-01-16 ... 2018-01-26T23:00:00
  * lat                    (lat) float32 45.0 44.75 44.5 ... 35.5 35.25 35.0
  * lon                    (lon) float32 325.0 325.25 325.5 ... 339.75 340.0
  * lev                    (lev) float32 1.0 2.0 3.0 4.0 ... 135.0 136.0 137.0
Data variables:
    TMP                    (time, lev, lat, lon) float32 ...
    SPFH                   (time, lev, lat, lon) float32 1.9870063e-06 ... 0.0055470048
    UGRD                   (time, lev, lat, lon) float32 ...
    VGRD                   (time, lev, lat, lon) float32 ...
    GP                     (time, lat, lon) float32 -6.565628 ... -2.815628
    NLPRES                 (time, lat, lon) float32 11.549789 ... 11.550724
    initial_time0_encoded  (time) float64 ...
    initial_time0          (time) |S18 ...
    height                 (lev, time, lat, lon) float64 7

## Optionally save the preprocessed file

In [12]:
save_preprocessed_era_ds = True
encoding = {'zlib': True, 'complevel': 6}
if save_preprocessed_era_ds:
    ds_era.to_netcdf(output_file_name,
                     encoding={v: encoding for v in ds_era})